In [ ]:
import numpy as np
import napari
import cellpose.models
from cellpose import io as cellpose_io
from cellpose import models


In [ ]:
import cellpose.models


In [6]:
import tifffile
file = tifffile.imread(r"C:\Users\micha\Desktop\MM_data_Pipeline_results_all\MM_pipeline_results_converted_Z1\Cropped_traps_directory\trap_1_all_frames_series.tiff")
file = file[20, 0, :, :]
print(file.shape)

(2035, 252)


### Run Cellpose and save masks

In [ ]:
import numpy as np
import tifffile
from cellpose import models, plot
import matplotlib.pyplot as plt

# Replace 'your_image_path' with the actual path to your image
file = tifffile.imread(r"C:\Users\micha\Desktop\MM_data_Pipeline_results_all\MM_pipeline_results_converted_Z1\Cropped_traps_directory\trap_1_all_frames_series.tiff")
file = file[20, 0, :, :]
print(file.shape)

model = models.CellposeModel(model_type='livecell_cp3')

masks, flows, styles = model.eval(file, diameter=90, channels=[0, 0])

fig = plt.figure(figsize=(12, 5))

# Replace 'your_channel_info' with the correct channel information


plot.show_segmentation(fig, file, masks, flows[0])
plt.tight_layout()
plt.show()
